In [7]:
%load_ext autoreload
%autoreload 2

import sys
import os
notebook_dir = os.path.dirname(os.path.abspath('__file__'))
project_root = os.path.abspath(os.path.join(notebook_dir, '..'))
sys.path.append(project_root)


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# 1. Sample Code

In [ ]:
from openai import OpenAI
from pydantic import BaseModel
import instructor


class UserDetail(BaseModel):
    name: str
    age: int


# enables `response_model` in create call
client = instructor.patch(
    OpenAI(
        base_url="http://localhost:11434/v1",
        api_key="ollama",  # required, but unused
    ),
    mode=instructor.Mode.JSON,
)

user = client.chat.completions.create(
    model="codegemma",
    messages=[
        {
            "role": "user",
            "content": "Jason is 30 years old",
        }
    ],
    response_model=UserDetail,
)

print(user)

# 2. Using GeoProcessing Model

In [3]:
import time
from openai import OpenAI
from pydantic import BaseModel
import instructor

# Patch the OpenAI client with instructor
client = instructor.patch(
    OpenAI(
        base_url="http://localhost:11434/v1",
        api_key="ollama",  # required, but unused
    ),
    mode=instructor.Mode.JSON,
)


In [ ]:
from geoprocessing_pipeline.models import GeoprocessingPipeline

# Define the function with the time calculation
def run_geo_processing_pipeline():
    # Record start time
    start_time = time.time()
    
    geo = client.chat.completions.create(
        model="codellama",
        messages=[
            {
              "role": "system",
              "content": """You are a GIS expert assistant. Here are the available GIS features you can use in this session:
      - **loadOsmData**: Load OpenStreetMap data for a specified geographical location.
        - Input: address (e.g., 'Kathmandu, Nepal'), filepath (always stored in 'data/').
      - **generateIsochrone**: Create an isochrone analysis for a given distance or time from a central point.
        - Input: distance (in meters) or time (in minutes) to generate a travel buffer around a point.
      - **filterPoints**: Apply attribute-based filtering on points, such as height, population, or other attributes.
        - Input: attribute, operator (e.g., '>', '<', '='), value.
      - **checkPointsWithinIsochrone**: Identify which points fall within a given isochrone buffer.
      - **loadData**: Load geospatial data of a specified type (e.g., points, polygons, roads).
        - Input: dataType (e.g., 'points', 'roads', 'buildings')
 
        """
            },
            {
              "role":"system",
                "content":"You need to create the steps, to process the query and take output of one function as input of another function. Always process the user request using only the required set of necessary functions."
            },
            {
                "role": "user",
                "content": "Find points where i can reach in 20 minutes from kathmandu nepal",
            }
        ],
        response_model=GeoprocessingPipeline,
    )

    # Record end time
    end_time = time.time()
    
    # Calculate the duration and print the running time
    duration = end_time - start_time
    print(f"Processing completed in {duration:.2f} seconds.")
    return geo


geo = run_geo_processing_pipeline()

In [ ]:
from pprint import pprint
pprint(geo, depth=2,indent=4)

In [ ]:
json_data = json.dumps(geo, default=lambda o: o.dict())
pprint(json_data, depth=1,indent=1)
print(json_data)

# 2. Running the logic

In [ ]:
import json
from geoprocessing_pipeline.data_loader import load_or_download_graph, load_data_by_type
from geoprocessing_pipeline.isochrone import generate_isochrone
from geoprocessing_pipeline.filter import filter_points_by_complex_query, filter_points_within_isochrone
from geoprocessing_pipeline.models import GeoprocessingPipeline, OsmDataInput
from pydantic import ValidationError

def run_geoprocessing_pipeline(json_data):
    """
    Runs the geoprocessing pipeline based on a JSON configuration.

    Parameters:
    - json_data (dict or str): A dictionary or JSON string representing the JSON configuration for the pipeline.

    Returns:
    - dict: A dictionary containing the outputs of the various pipeline steps.
    """
    # If json_data is a string, parse it into a dictionary
    if isinstance(json_data, str):
        try:
            json_data = json.loads(json_data)
        except json.JSONDecodeError as e:
            raise ValueError(f"Invalid JSON string: {e}")

    # This dictionary will hold the output of each function in the pipeline
    outputs = {}

    # Validate and parse the JSON configuration using the pydantic model
    try:
        pipeline = GeoprocessingPipeline(**json_data)
    except ValidationError as e:
        raise ValueError(f"Invalid pipeline configuration: {e}")

    # Iterate through each function step in the pipeline configuration
    for func in pipeline.functions:
        func_name = func.functionName

        # Load or download OSM data
        if func_name == "loadOsmData":
            # Check if input data is a reference (string) or a full OsmDataInput object
            if isinstance(func.input.data, str):
                # If it's a reference, retrieve the actual data from the outputs dictionary
                osm_data_input = outputs[func.input.data]
            else:
                # Otherwise, it's a direct OsmDataInput object
                osm_data_input = func.input.data

            address = osm_data_input
            filepath = osm_data_input.filepath
            output = load_or_download_graph(address, filepath)
        
        # Generate Isochrone
        elif func_name == "generateIsochrone":
            # Retrieve the OSM network from the outputs (data is a reference key here)
            osm_network = outputs[func.input.data]
            distance = func.input.parameters.distance
            output = generate_isochrone(osm_network, distance)
        
        # Load generic data (points, buildings, etc.)
        elif func_name == "loadData":
            data_type = func.input.parameters.dataType
            output = load_data_by_type(data_type)
        
        # Filter Points by Complex Query (e.g., height > 20)
        elif func_name == "filterPoints" and isinstance(func.input.parameters, FilterCriteria):
            points = outputs[func.input.data]  # Use the points that were loaded earlier
            attribute = func.input.parameters.attribute
            operator = func.input.parameters.operator
            value = func.input.parameters.value
            output = filter_points_by_complex_query(points, attribute, operator, value)
        
        # Check which points are within the isochrone
        elif func_name == "checkPointsWithinIsochrone":
            points = outputs[func.input.data]
            isochrone_polygon = outputs[func.input.parameters.isochrone]
            output = filter_points_within_isochrone(points, isochrone_polygon)

        # Store the output of this function in the outputs dictionary
        outputs[func.output] = output

    return outputs


In [ ]:
json_data_input = json.loads(json_data)
outputs = run_geoprocessing_pipeline(json_data_input)